In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import IPython.display as ipd
import matplotlib.pyplot as plt
import torch
import torchutil
import promonet

In [ ]:
# Get distribution without augmentation or variable-width
promonet.AUGMENT_PITCH = False
promonet.AUGMENT_LOUDNESS = False
allpitch = []
lowstems = []
dataset = promonet.data.Dataset('vctk', 'train')
for stem in torchutil.iterator(
    dataset.stems,
    'promonet.load.pitch_distribution'
):
    pitch = torch.load(
        dataset.cache / f'{stem}-viterbi-pitch.pt')
    periodicity = torch.load(
        dataset.cache / f'{stem}-viterbi-periodicity.pt')
    allpitch.append(
        pitch[periodicity > promonet.VOICING_THRESHOLD])
    if torch.isnan(pitch).any():
        print('nan')
    if pitch.min() < 65.:
        lowstems.append(stem)
hz, _ = torch.sort(torch.cat(allpitch))
hz = torch.clip(hz, promonet.FMIN, promonet.FMAX)
logfmin = torch.log2(torch.tensor(promonet.FMIN))
logfmax = torch.log2(torch.tensor(promonet.FMAX))
centered = torch.log2(hz) - logfmin
normalized = centered / (logfmax - logfmin)
original_bins = ((promonet.PITCH_BINS - 1) * normalized).to(torch.long)
indices = torch.linspace(
    0.,
    len(hz) - 1,
    promonet.PITCH_BINS,
    dtype=torch.float64
).to(torch.long)
original_distribution = hz[indices]
original_distribution[0] = promonet.FMIN

In [ ]:
# Get distribution with augmentation
promonet.AUGMENT_PITCH = True
promonet.AUGMENT_LOUDNESS = True
allpitch = []
dataset = promonet.data.Dataset('vctk', 'train')
for stem in torchutil.iterator(
    dataset.stems,
    'promonet.load.pitch_distribution'
):
    pitch = torch.load(
        dataset.cache / f'{stem}-viterbi-pitch.pt')
    periodicity = torch.load(
        dataset.cache / f'{stem}-viterbi-periodicity.pt')
    allpitch.append(
        pitch[torch.logical_and(~torch.isnan(pitch), periodicity > promonet.VOICING_THRESHOLD)])
hz, _ = torch.sort(torch.cat(allpitch))
hz = torch.clip(hz, promonet.FMIN, promonet.FMAX)
logfmin = torch.log2(torch.tensor(promonet.FMIN))
logfmax = torch.log2(torch.tensor(promonet.FMAX))
centered = torch.log2(hz) - logfmin
normalized = centered / (logfmax - logfmin)
augmented_bins = ((promonet.PITCH_BINS - 1) * normalized).to(torch.long)
indices = torch.linspace(
    len(hz) / promonet.PITCH_BINS,
    len(hz) - 1,
    promonet.PITCH_BINS,
    dtype=torch.float64
).to(torch.long)
augmented_distribution = hz[indices]

In [ ]:
variable_bins = torch.clip(torch.searchsorted(hz[indices], hz), 0, promonet.PITCH_BINS - 1)

In [ ]:
original_counts = torch.bincount(original_bins, minlength=promonet.PITCH_BINS)
augmented_counts = torch.bincount(augmented_bins, minlength=promonet.PITCH_BINS)
variable_counts = torch.bincount(variable_bins, minlength=promonet.PITCH_BINS)
original_normalized = original_counts / sum(original_counts)
augmented_normalized = augmented_counts / sum(augmented_counts)
variable_normalized = variable_counts / sum(variable_counts)
ymax = max(variable_normalized.max(), augmented_normalized.max(), original_normalized.max())

In [ ]:
def entropy(d):
    return - (d * torch.log(d + 1e-9)).sum()
original_entropy = entropy(original_normalized)
augmented_entropy = entropy(augmented_normalized)
variable_entropy = entropy(variable_normalized)

In [ ]:
xlabels = torch.tensor([50, 100, 200, 400])
original_xticks = ((promonet.PITCH_BINS - 1) * (torch.log2(xlabels) - promonet.LOG_FMIN) / (promonet.LOG_FMAX - promonet.LOG_FMIN)).to(torch.long)
augmented_xticks = original_xticks
variable_xticks = torch.clip(torch.searchsorted(hz[indices], xlabels), 0, promonet.PITCH_BINS - 1)
xlabels = xlabels.tolist()
variable_xlabels = xlabels.copy()
variable_xlabels[-1] = f'{xlabels[-1]} ' + r'$\bf{Hz}$'

In [ ]:
figure, axes = plt.subplots(1, 3, figsize=(15, 3))
axes[0].stairs(original_counts, color='black')
axes[0].set_xlabel(r'$\bf{Original}$' + f'\n (Entropy = {original_entropy:02f})', fontsize=12)
axes[0].set_xticks(original_xticks, xlabels)
axes[0].spines['top'].set_visible(False)
axes[0].spines['right'].set_visible(False)
axes[0].spines['bottom'].set_visible(False)
axes[0].spines['left'].set_visible(False)
axes[0].get_yaxis().set_ticks([])
axes[0].set_ylim(0., ymax * original_counts.sum())
for tick in original_xticks:
    axes[0].vlines(tick, 0, ymax * original_counts.sum(), color='black', linestyle='--', alpha=.4)
axes[1].stairs(augmented_counts, color='black')
# axes[1].set_title('Maximizing the entropy of the pitch training distribution', weight='bold', fontsize=14)
axes[1].set_xlabel(r'$\bf{Augmented}$' + f'\n (Entropy = {augmented_entropy:02f})', fontsize=12)
axes[1].set_xticks(augmented_xticks, xlabels)
axes[1].spines['top'].set_visible(False)
axes[1].spines['right'].set_visible(False)
axes[1].spines['bottom'].set_visible(False)
axes[1].spines['left'].set_visible(False)
axes[1].get_yaxis().set_ticks([])
axes[1].set_ylim(0., ymax * augmented_counts.sum())
for tick in augmented_xticks:
    axes[1].vlines(tick, 0, ymax * augmented_counts.sum(), color='black', linestyle='--', alpha=.4)
axes[2].set_xlabel(r'$\bf{Variable}$-$\bf{width}$' + f'\n (Entropy = {variable_entropy:02f})', fontsize=12)
axes[2].set_xticks(variable_xticks, variable_xlabels)
axes[2].stairs(variable_counts, color='black')
axes[2].spines['top'].set_visible(False)
axes[2].spines['right'].set_visible(False)
axes[2].spines['bottom'].set_visible(False)
axes[2].spines['left'].set_visible(False)
axes[2].get_yaxis().set_ticks([])
axes[2].set_ylim(0., ymax * variable_counts.sum())
for tick in variable_xticks:
    axes[2].vlines(tick, 0, ymax * variable_counts.sum(), color='black', linestyle='--', alpha=.4)
figure.savefig('pitch-entropy.pdf', bbox_inches='tight', pad_inches=0, dpi=300)
figure.savefig('pitch-entropy.png', bbox_inches='tight', pad_inches=0, dpi=300, transparent=True)